In [ ]:
#Imports
import numpy as np
import tensorflow as tf
# Model/layer imports
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [ ]:
# #If using GPU, reset memory allocation, and assign GPU with memory growth.
# tf.compat.v1.reset_default_graph()
# physical_devices = tf.config.list_physical_devices('GPU')
# tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

In [ ]:
#Open Shakespeare.txt
path_to_file = "shakesbot.txt"
text = open(path_to_file, 'r').read()

In [ ]:
#Obtain unique vocab set and size 
vocab = sorted(set(text))
vocab_size = len(vocab)

In [ ]:
#Create an index mapping for characters in vocab (bi-directional)
char_to_ind = {char:ind for ind, char in enumerate(vocab)}
ind_to_char = np.array(vocab)

In [ ]:
#Encode whole of text file using vocab index map
encoded_text = np.array([char_to_ind[c] for c in text])

In [ ]:
# #Query line(s) length(s)
# line = 'From fairest creatures we desire increase'
# print(len(line))

# lines='''
#   From fairest creatures we desire increase,
#   That thereby beauty's rose might never die,
#   But as the riper should by time decease,
#   His tender heir might bear his memory:
#   '''
# print(len(lines))

In [ ]:
#Set length of sequence
seq_len = 150

In [ ]:
#Determine total number of sequences in text
total_num_seq = len(text) // (seq_len+1)

In [ ]:
#Create a Character Dataset from encoded text 
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

In [ ]:
#Combine into sequences
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [ ]:
# Create method for splitting sequences into input/target 
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

#Use method to create complete training dataset
dataset = sequences.map(create_seq_targets)

In [ ]:
#Set batch size
batch_size = 64

In [ ]:
#Shuffle the dataset
buffer_size=10000
dataset = dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [ ]:
#Set embedding dimensions
embed_dim = 256

In [ ]:
#Choose amount of neurons in LSTM layer
rnn_neurons = 1024

In [ ]:
#Customised Sparse_cat_cross loss function 
def sparse_cat_loss(y_true,y_pred):
    return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [ ]:
#Function for creating model
def create_model(vocab_size, embed_dim,rnn_neurons, batch_size):
    model = Sequential()
    model.add(Embedding(vocab_size, embed_dim, batch_input_shape=[batch_size,None]))
    model.add(LSTM(rnn_neurons, stateful=True, return_sequences=True, recurrent_initializer='glorot_uniform', batch_input_shape=[batch_size,None, embed_dim]))
    model.add(Dropout(0.2))
    model.add(Dense(vocab_size))
    model.compile('adam', loss=sparse_cat_loss)
    return model

In [ ]:
# Create training model
model = create_model(vocab_size=vocab_size,
                     embed_dim=embed_dim,
                     rnn_neurons=rnn_neurons,
                     batch_size=batch_size)

In [ ]:
# Display model summary
model.summary()

In [ ]:
#Set epochs 
epochs = 100

In [ ]:
#Create early stopping function
early_stop = EarlyStopping(monitor='loss', patience=3)

In [ ]:
#Fit the model
model.fit(dataset,epochs=epochs,callbacks=[early_stop])

In [ ]:
#Save the model
model.save('shakesbot_train.h5')

In [ ]:
#Create a new version of the model (test_model) with a single input batch size
test_model = create_model(vocab_size,embed_dim,rnn_neurons,batch_size=1)
test_model.load_weights('shakesbot_train.h5')
test_model.build(tf.TensorShape([1,None]))
test_model.save('shakesbot_test.h5')

In [ ]:
# Display test model summary
test_model.summary()

In [ ]:
###RUN THE BELOW CODE IF YOU ARE OPENING THIS AFTER TRAINING###

In [3]:
# #If loading seperately
# import tensorflow as tf
# from tensorflow.keras.models import load_model
# import numpy as np

# # #if using GPU, set the correct GPU and assign memory growth
# # tf.compat.v1.reset_default_graph()
# # physical_devices = tf.config.list_physical_devices('GPU')
# # tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

# #Open shakesbot.txt
# path_to_file = "shakesbot.txt"
# text = open(path_to_file, 'r').read()

# #Obtain unique vocab set and size 
# vocab = sorted(set(text))
# vocab_size = len(vocab)

# #Create an index mapping for characters in vocab (bi-directional)
# char_to_ind = {char:ind for ind, char in enumerate(vocab)}
# ind_to_char = np.array(vocab)

# #Load the model to memory
# test_model = load_model('shakesbot_test.h5', compile=False)

In [4]:
###RUN THE ABOVE CODE IF YOU ARE OPENING THIS AFTER TRAINING###

In [5]:
#Test function for generating consecutive text
def generate_text(model,start_seed,num_generate=500,temperature=1):
    
    #Map each character in start_seed to it's relative index
    input_eval = [char_to_ind[s] for s in start_seed]
    
    #Expand the dimensions
    input_eval = tf.expand_dims(input_eval, 0)
    
    #Create empty list for generated text
    text_generated = []
    
    #Reset the states of tyhe model        
    model.reset_states()
    
    #for each iteration of num_generate
    for i in range(num_generate):
        
        #Obtain probability matrix for current iteration 
        predictions = model(input_eval)
        
        #Reduce dimensions
        predictions = tf.squeeze(predictions,0)
        
        #Multiply probabily matrix by temperature
        predictions = predictions/temperature
        
        #Select a random outcome, based on the unnormalised log-probabilities produced by the model
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        #Expand dimensions of prediction and assign as next input evaluation
        input_eval = tf.expand_dims([predicted_id], 0)
        
        #Convert prediction to char and append to generated list of text
        text_generated.append(ind_to_char[predicted_id])
        
    #return the initial input, concatenated with the generated text. 
    return(start_seed+"".join(text_generated))

In [8]:
# Test the model with sample text
print(generate_text(model=test_model,start_seed="This is a test", num_generate=500, temperature=0.5))

This is a testimony of the plainer
    of such a paragon. I shall see him and be the face of a
    speech, and the players of their death shall lose thee. Go, fellow,
    and therefore am not so long as thou art. Do you not
    think there was not the rest of thee to knock'd me to
    his hands? Is he underneath the fall and cheerful rock?
    The deadly stockings of the Spite of Troy,
    With all the last of them and the adventure of the state,
    That should be sure to see me light and look
    To think t
